# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [7]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/jasmine/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jasmine/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jasmine/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
# load data from database

engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('MessageCategory', 'sqlite:///InsertDatabaseName.db')  


In [9]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2. Write a tokenization function to process your text data

In [39]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # tokenize text
    tokens = word_tokenize(text)
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [54]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))

        ])),

        ('clf', RandomForestClassifier())
        ])
"""
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4]
    }

cv = GridSearchCV(pipeline, param_grid=parameters)
"""

"\nparameters = {\n        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),\n        'clf__n_estimators': [50, 100, 200],\n        'clf__min_samples_split': [2, 3, 4]\n    }\n\ncv = GridSearchCV(pipeline, param_grid=parameters)\n"

In [55]:
df = df.dropna()

In [56]:
X = df['message']
Y = df['related']

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [57]:

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

model = pipeline # cv
model.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x121a208b0>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())]))])),
                ('clf', RandomForestClassifier())])

In [58]:
X_test

2882    Because we are hunger and weakness is killing ...
4514    PAM is distributing food but they said that it...
7719    I would like to have some informations about s...
33      People have been sleeping outdoors in a field ...
4924    in Tigoave we have lots of problems ,send help...
                              ...                        
2260    my house has collapsed im living in the street...
8924    4636 I NEED AN ANSWER ON MY SUBJECT,TELL ME WH...
6775                   information about the earthquake. 
37      I would like to get help at Cote Plage in Carr...
3979            NOTES: I THANK YOU FOR YOUR UNDERSTANDING
Name: message, Length: 2045, dtype: object

In [59]:
y_pred = model.predict(X_test)

In [65]:
len(X_test)

2045

In [67]:
X_test[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [63]:
len(y_pred)

2045

In [66]:
y_pred[0]

1.0

In [71]:
model.predict(['Weather update - a cold front from Cuba that could pass over Haiti'])[0] == 1

True

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [60]:
y_pred

array([1., 1., 1., ..., 1., 1., 1.])

In [61]:
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.38      0.11      0.17       679
     class 1       0.66      0.91      0.76      1333
     class 2       0.00      0.00      0.00        33

    accuracy                           0.63      2045
   macro avg       0.34      0.34      0.31      2045
weighted avg       0.55      0.63      0.55      2045



### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [125]:
import pickle
# create an iterator object with write permission - model.pkl
with open('model_pkl', 'wb') as files:
    pickle.dump(model, files)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [4]:
create_engine("sqlite:///" + 'DisasterResponse.db')

Engine(sqlite:///DisasterResponse.db)

In [ ]:
dict(zip(df.columns[4:], classification_labels))

In [83]:
dict(zip(df.columns[4], [1]))

{'r': 1}

In [87]:
model.predict(['Tianmin'])[0]

1.0

In [82]:
df.columns[4]

'related'